# התאמת מחרוזות

מחברת אינטראקטיבית לאלגוריתמי חיפוש תבנית: Naive, Rabin-Karp, KMP.

## 1. חיפוש נאיבי

In [ ]:
def naive_search(text, pattern):
    """
    חיפוש נאיבי - O(n*m)
    מחזיר רשימת מיקומים
    """
    n, m = len(text), len(pattern)
    matches = []
    comparisons = 0
    
    for i in range(n - m + 1):
        match = True
        for j in range(m):
            comparisons += 1
            if text[i + j] != pattern[j]:
                match = False
                break
        if match:
            matches.append(i)
    
    return matches, comparisons

# דוגמאות
text = "AABAACAADAABAABA"
pattern = "AABA"

matches, comps = naive_search(text, pattern)
print(f"טקסט: {text}")
print(f"תבנית: {pattern}")
print(f"מיקומים: {matches}")
print(f"השוואות: {comps}")

## 2. ויזואליזציה של החיפוש

In [ ]:
def visualize_naive(text, pattern):
    """הדגמה ויזואלית של החיפוש הנאיבי"""
    n, m = len(text), len(pattern)
    
    print(f"Text:    {text}")
    print(f"Pattern: {pattern}")
    print("=" * 40)
    
    for i in range(n - m + 1):
        # הצג יישור נוכחי
        alignment = ' ' * i + pattern
        match = text[i:i+m] == pattern
        status = "✓ MATCH" if match else "✗"
        print(f"i={i:2}: {' ' * i}{pattern} {status}")
    
    print("=" * 40)

visualize_naive("ABCABCD", "BCD")

## 3. Rabin-Karp עם Rolling Hash

In [ ]:
def rabin_karp(text, pattern, base=256, prime=101):
    """
    Rabin-Karp עם rolling hash
    זמן ממוצע: O(n+m), worst case: O(nm)
    """
    n, m = len(text), len(pattern)
    matches = []
    
    if m > n:
        return matches
    
    # חישוב hash של התבנית ו-hash ראשוני של הטקסט
    pattern_hash = 0
    text_hash = 0
    h = pow(base, m - 1, prime)  # המקדם הגבוה ביותר
    
    # חישוב hash ראשוני
    for i in range(m):
        pattern_hash = (pattern_hash * base + ord(pattern[i])) % prime
        text_hash = (text_hash * base + ord(text[i])) % prime
    
    # סריקת הטקסט
    for i in range(n - m + 1):
        # בדיקת hash
        if pattern_hash == text_hash:
            # וודא התאמה (בגלל התנגשויות)
            if text[i:i+m] == pattern:
                matches.append(i)
        
        # rolling hash - עדכון לחלון הבא
        if i < n - m:
            text_hash = ((text_hash - ord(text[i]) * h) * base + ord(text[i + m])) % prime
            if text_hash < 0:
                text_hash += prime
    
    return matches

# דוגמה
text = "AABAACAADAABAABA"
pattern = "AABA"

matches = rabin_karp(text, pattern)
print(f"טקסט: {text}")
print(f"תבנית: {pattern}")
print(f"מיקומים: {matches}")

## 4. הדגמת Rolling Hash

In [ ]:
def rolling_hash_demo(text, window_size, base=10, prime=97):
    """הדגמת חישוב rolling hash"""
    n, m = len(text), window_size
    h = pow(base, m - 1, prime)
    
    # hash ראשוני
    current_hash = 0
    for i in range(m):
        current_hash = (current_hash * base + ord(text[i])) % prime
    
    print(f"Text: {text}, Window size: {m}")
    print(f"Base: {base}, Prime: {prime}")
    print("=" * 50)
    
    for i in range(n - m + 1):
        window = text[i:i+m]
        print(f"i={i}: '{window}' -> hash = {current_hash}")
        
        # rolling
        if i < n - m:
            old_char = text[i]
            new_char = text[i + m]
            current_hash = ((current_hash - ord(old_char) * h) * base + ord(new_char)) % prime
            if current_hash < 0:
                current_hash += prime

rolling_hash_demo("ABCDEF", 3)

## 5. KMP - בניית טבלת Failure

In [ ]:
def build_failure_table(pattern):
    """
    בניית טבלת failure עבור KMP
    failure[i] = אורך הרישא הארוכה ביותר שהיא גם סיפא של pattern[:i+1]
    """
    m = len(pattern)
    failure = [0] * m
    
    j = 0  # אורך הרישא הקודמת
    
    for i in range(1, m):
        # חפש רישא מתאימה
        while j > 0 and pattern[i] != pattern[j]:
            j = failure[j - 1]
        
        if pattern[i] == pattern[j]:
            j += 1
        
        failure[i] = j
    
    return failure

# דוגמאות
patterns = ["ABABAC", "AABAA", "ABCABC", "AAAA"]

for p in patterns:
    f = build_failure_table(p)
    print(f"Pattern: {p}")
    print(f"Index:   {list(range(len(p)))}")
    print(f"Failure: {f}")
    print()

## 6. KMP - חיפוש

In [ ]:
def kmp_search(text, pattern):
    """
    KMP - Knuth-Morris-Pratt
    O(n + m)
    """
    n, m = len(text), len(pattern)
    matches = []
    comparisons = 0
    
    if m == 0:
        return matches
    
    # בניית טבלת failure
    failure = build_failure_table(pattern)
    
    j = 0  # מיקום בתבנית
    
    for i in range(n):
        # חפש התאמה
        while j > 0 and text[i] != pattern[j]:
            comparisons += 1
            j = failure[j - 1]
        
        comparisons += 1
        if text[i] == pattern[j]:
            j += 1
        
        # מצאנו התאמה
        if j == m:
            matches.append(i - m + 1)
            j = failure[j - 1]
    
    return matches, comparisons

# דוגמה
text = "AABAACAADAABAABA"
pattern = "AABA"

matches, comps = kmp_search(text, pattern)
print(f"טקסט: {text}")
print(f"תבנית: {pattern}")
print(f"מיקומים: {matches}")
print(f"השוואות: {comps}")

## 7. השוואת ביצועים

In [ ]:
import time

def benchmark(text, pattern):
    """השוואת אלגוריתמים"""
    print(f"Text length: {len(text)}, Pattern length: {len(pattern)}")
    print("=" * 50)
    
    # Naive
    start = time.time()
    matches1, comps1 = naive_search(text, pattern)
    time1 = time.time() - start
    print(f"Naive:      {len(matches1)} matches, {comps1} comparisons, {time1*1000:.2f}ms")
    
    # Rabin-Karp
    start = time.time()
    matches2 = rabin_karp(text, pattern)
    time2 = time.time() - start
    print(f"Rabin-Karp: {len(matches2)} matches, {time2*1000:.2f}ms")
    
    # KMP
    start = time.time()
    matches3, comps3 = kmp_search(text, pattern)
    time3 = time.time() - start
    print(f"KMP:        {len(matches3)} matches, {comps3} comparisons, {time3*1000:.2f}ms")

# מקרה ממוצע
print("\n=== מקרה ממוצע ===")
text = "The quick brown fox jumps over the lazy dog" * 100
benchmark(text, "the")

# מקרה גרוע (הרבה התאמות חלקיות)
print("\n=== מקרה גרוע ===")
text = "A" * 1000
benchmark(text, "AAAA")

## 8. סיכום

| אלגוריתם | זמן ריצה | הערות |
|----------|----------|-------|
| Naive | O(nm) | פשוט, טוב למקרים קטנים |
| Rabin-Karp | O(n+m) avg, O(nm) worst | rolling hash, טוב לחיפוש מספר תבניות |
| KMP | O(n+m) | failure table, מובטח לינארי |